## Prix Spot

In [1]:
# Librairies nécessaires
from google.cloud import bigquery
import os
import pandas as pd
from datetime import datetime
import pytz

In [2]:
# Chemin d'accès
donnees = r"C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\Soutirage\forecast-perimetre-TR\Output"
dossier = r"C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation"
output = r"C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation\Output"

# Meryam GHASSIR

In [3]:
# Récupérer les données de Prix Spot depuis BigQuery (€/MWh)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation\credentials.json"

def get_bigquery_client(table_id):
    client = bigquery.Client()
    table = client.get_table(table_id)
    df = client.list_rows(table).to_dataframe()
    return df

In [4]:
table_spot = "gcp-data-energy-11459.RTE_Data.spot-prices"
df_spot = get_bigquery_client(table_spot)

# Ordonner les données par ordre croissant chronologique
df_spot['Timestamp'] = pd.to_datetime(df_spot['Timestamp'], format='%Y-%m-%d %H:%M:%S', utc=True)
df_spot = df_spot.sort_values(by='Timestamp', ascending=True)
df_spot.rename(columns={'Timestamp' : 'date'}, inplace=True)
df_spot.set_index('date', inplace=True)

paris_tz = pytz.timezone("Europe/Paris")
df_spot.index = df_spot.index.tz_convert(paris_tz)

df_spot.index = pd.to_datetime(df_spot.index).tz_localize(None)

print(df_spot.tail())

C:\Users\Meryam GHASSIR\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


                      Price
date                       
2025-09-09 19:00:00   90.00
2025-09-09 20:00:00  111.37
2025-09-09 21:00:00   91.60
2025-09-09 22:00:00   95.49
2025-09-09 23:00:00   78.37


In [5]:
# Vérification de valeurs manquantes dans df_spot

date_range = pd.date_range(start=df_spot.index.min(), end=df_spot.index.max(), freq='h')
missing_dates = date_range.difference(df_spot.index)
print(f"Nb dates manquantes dans df_spot : {len(missing_dates)}")
if len(missing_dates) > 0:
    print("Dates manquantes :", missing_dates)

df_missing_dates = pd.DataFrame({'Missing Date': missing_dates})
df_missing_dates['Missing Date'] = df_missing_dates['Missing Date'].dt.tz_localize(None)

print(df_missing_dates.head())
df_missing_dates.to_csv("dates_manquantes_spot.csv", index=False, sep=";")

Nb dates manquantes dans df_spot : 332
Dates manquantes : DatetimeIndex(['2008-03-30 02:00:00', '2009-03-29 02:00:00',
               '2010-03-28 02:00:00', '2011-03-27 02:00:00',
               '2012-03-25 02:00:00', '2013-03-31 02:00:00',
               '2014-03-30 02:00:00', '2015-03-29 02:00:00',
               '2016-03-27 02:00:00', '2017-03-26 02:00:00',
               ...
               '2025-09-05 14:00:00', '2025-09-05 15:00:00',
               '2025-09-05 16:00:00', '2025-09-05 17:00:00',
               '2025-09-05 18:00:00', '2025-09-05 19:00:00',
               '2025-09-05 20:00:00', '2025-09-05 21:00:00',
               '2025-09-05 22:00:00', '2025-09-05 23:00:00'],
              dtype='datetime64[ns]', length=332, freq=None)
         Missing Date
0 2008-03-30 02:00:00
1 2009-03-29 02:00:00
2 2010-03-28 02:00:00
3 2011-03-27 02:00:00
4 2012-03-25 02:00:00


In [6]:
# df_spot.rename(columns={'Timestamp':'date'}, inplace=True)
# df_spot['date'] = pd.to_datetime(df_spot['date'])        
df_spot.index = df_spot.index.tz_localize(None)   
# df_spot.set_index('date', inplace=True)
df_spot.sort_values(by='date', inplace=True)
print(df_spot.head())

                     Price
date                      
2008-01-01 01:00:00  71.19
2008-01-01 02:00:00  68.94
2008-01-01 03:00:00  62.92
2008-01-01 04:00:00  47.98
2008-01-01 05:00:00  44.99


In [ ]:
# Sauverage du DataFrame df_spot dans un fichier CSV (si pas de données manquantes)
df_spot.to_csv("prix_spot.csv", index=False, sep=";")

**On s'arrête à cette étape du code s'il n'y a pas de de données manquantes.**

In [7]:
# Créer les dates manquantes dans df_spot en important le fichier des dates manquantes
df_missing_dates = pd.read_csv("dates_manquantes_spot.csv", sep=";")
df_missing_dates['Missing Date'] = pd.to_datetime(df_missing_dates['Missing Date'])

# Harmoniser l'index de df_spot (retirer la timezone si présente)
df_spot.index = pd.to_datetime(df_spot.index)
if hasattr(df_spot.index, 'tz'):
    df_spot.index = df_spot.index.tz_localize(None)

# Ajout des lignes pour les dates manquantes dans df_spot avec valeurs NaN
for missing_date in df_missing_dates['Missing Date']:
    if missing_date not in df_spot.index:
        df_spot.loc[missing_date] = None  # Ajouter une ligne avec la date manquante et des valeurs NaN

df_spot.sort_index(inplace=True)
print(df_spot.tail())

                      Price
date                       
2025-09-09 19:00:00   90.00
2025-09-09 20:00:00  111.37
2025-09-09 21:00:00   91.60
2025-09-09 22:00:00   95.49
2025-09-09 23:00:00   78.37


In [8]:
# Vérification des dates manquantes dans df_spot après ajout

date_range = pd.date_range(start=df_spot.index.min(), end=df_spot.index.max(), freq='h')
missing_dates = date_range.difference(df_spot.index)
print(f"Nb dates manquantes dans df_spot : {len(missing_dates)}")
if len(missing_dates) > 0:
    print("Dates manquantes :", missing_dates)

df_missing_dates = pd.DataFrame({'Missing Date': missing_dates})
print(df_missing_dates.head())
df_missing_dates.to_csv("dates_manquantes_spot.csv", index=False, sep=";")

# Nombre de valeurs NaN dans df_spot
# On doit avoir le même nombre de valeurs NaN que de dates manquantes
nan_count = df_spot.isna().sum().sum()
print(f"Nombre de valeurs NaN dans df_spot : {nan_count}")

Nb dates manquantes dans df_spot : 0
Empty DataFrame
Columns: [Missing Date]
Index: []
Nombre de valeurs NaN dans df_spot : 332


In [9]:
# On récupère les données de prix spot depuis le fichier EPEX FR 
df_epex = pd.read_csv(r"C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\HPFC\EPEX_FR.csv", sep=";") #, parse_dates=['date'], index_col='date')
print(df_epex.tail())

# Harmoniser la colonne date qui est au format 'dd/mm/yyyy hh:mm' et la convertir en format datetime 'yyyy-mm-dd hh:mm:ss'
df_epex['date'] = pd.to_datetime(df_epex['date']) # df_epex['date'] = pd.to_datetime(df_epex['date'], format='%d/%m/%Y %H:%M.') # IGNORE
df_epex.set_index('date', inplace=True)

df_epex.rename(columns={'Prix Spot': 'Price'}, inplace=True)
df_epex.sort_values(by='date', inplace=True)

print(df_epex.tail())

                       date  Prix Spot
154532  2025-08-22 19:00:00      84.56
154533  2025-08-22 20:00:00     104.53
154534  2025-08-22 21:00:00     103.49
154535  2025-08-22 22:00:00     101.59
154536  2025-08-22 23:00:00      94.37
                      Price
date                       
2025-08-22 19:00:00   84.56
2025-08-22 20:00:00  104.53
2025-08-22 21:00:00  103.49
2025-08-22 22:00:00  101.59
2025-08-22 23:00:00   94.37


In [10]:
# Dates manquantes dans df_epex (contrôle dates manquantes dans le fichier EPEX FR pour voir si on peut effectivement utiliser 
# ce fichier pour remplacer les NaN dans df_spot)
date_range_epex = pd.date_range(start=df_epex.index.min(), end=df_epex.index.max(), freq='h')
missing_dates_epex = date_range_epex.difference(df_epex.index)
print(f"Nb dates manquantes dans df_epex : {len(missing_dates_epex)}")
if len(missing_dates_epex) > 0:
    print("Dates manquantes dans df_epex :", missing_dates_epex)

Nb dates manquantes dans df_epex : 123
Dates manquantes dans df_epex : DatetimeIndex(['2022-03-27 02:00:00', '2024-03-31 02:00:00',
               '2025-03-30 02:00:00', '2025-08-05 00:00:00',
               '2025-08-05 01:00:00', '2025-08-05 02:00:00',
               '2025-08-05 03:00:00', '2025-08-05 04:00:00',
               '2025-08-05 05:00:00', '2025-08-05 06:00:00',
               ...
               '2025-08-17 14:00:00', '2025-08-17 15:00:00',
               '2025-08-17 16:00:00', '2025-08-17 17:00:00',
               '2025-08-17 18:00:00', '2025-08-17 19:00:00',
               '2025-08-17 20:00:00', '2025-08-17 21:00:00',
               '2025-08-17 22:00:00', '2025-08-17 23:00:00'],
              dtype='datetime64[ns]', length=123, freq=None)


In [40]:
# On remplace les valeurs NaN de df_spot par les valeurs correspondantes de df_epex

# Supprimer les doublons d'index en gardant la première occurrence
df_spot = df_spot[~df_spot.index.duplicated(keep='first')]
df_epex = df_epex[~df_epex.index.duplicated(keep='first')]

# Remplacer les valeurs NaN dans df_spot par les valeurs correspondantes de df_epex
df_spot['Price'] = df_spot['Price'].combine_first(df_epex['Price'])

nan_spot = df_spot[df_spot['Price'].isna()]

# Vérifier le résultat
print(df_spot[df_spot['Price'].isna()])  # Affiche les lignes où il reste des NaN
print(f"Nombre de valeurs NaN dans df_spot après remplacement : {df_spot['Price'].isna().sum()}")

                     Price
date                      
2022-03-27 02:00:00    NaN
2024-03-31 02:00:00    NaN
2025-03-30 02:00:00    NaN
2025-08-05 00:00:00    NaN
2025-08-05 01:00:00    NaN
...                    ...
2025-08-17 19:00:00    NaN
2025-08-17 20:00:00    NaN
2025-08-17 21:00:00    NaN
2025-08-17 22:00:00    NaN
2025-08-17 23:00:00    NaN

[123 rows x 1 columns]
Nombre de valeurs NaN dans df_spot après remplacement : 123


In [41]:
# Ajout des dates manquantes de 2022-03-27 02:00:00 et 2024-03-31 02:00:00 (passage à l'heure d'été, conso = 0MWh)

# Mettre à 0 les valeurs manquantes du 2022-03-27 02:00:00 et du 2024-03-31 02:00:00
df_spot.loc[df_spot.index == pd.Timestamp('2022-03-27 02:00:00'), 'Price'] = 0.0
df_spot.loc[df_spot.index == pd.Timestamp('2024-03-31 02:00:00'), 'Price'] = 0.0
df_spot.loc[df_spot.index == pd.Timestamp('2025-03-30 02:00:00'), 'Price'] = 0.0

print(df_spot[df_spot['Price'].isna()]) 
print(f"Nombre de valeurs NaN dans df_spot après remplacement : {df_spot['Price'].isna().sum()}")

                     Price
date                      
2025-08-05 00:00:00    NaN
2025-08-05 01:00:00    NaN
2025-08-05 02:00:00    NaN
2025-08-05 03:00:00    NaN
2025-08-05 04:00:00    NaN
...                    ...
2025-08-17 19:00:00    NaN
2025-08-17 20:00:00    NaN
2025-08-17 21:00:00    NaN
2025-08-17 22:00:00    NaN
2025-08-17 23:00:00    NaN

[120 rows x 1 columns]
Nombre de valeurs NaN dans df_spot après remplacement : 120


In [42]:
# Retraitement exceptionnel : Suppression de toutes les dates à partir du 5 août (absence du 5, 6, 7, 10 et 17 août 2025)
df_spot = df_spot[df_spot.index < '2025-08-05']
print(df_spot[df_spot['Price'].isna()]) 
print(f"Nombre de valeurs NaN dans df_spot après remplacement : {df_spot['Price'].isna().sum()}")

Empty DataFrame
Columns: [Price]
Index: []
Nombre de valeurs NaN dans df_spot après remplacement : 0


In [13]:
# Retraitement exceptionnel pour les heures manquantes du 17 juillet 2025 (en attendant la régularisation - update : 04/08/2025 10h55)
# Suppression des lignes NaN du 17 juillet 2025
df_spot = df_spot[~df_spot.index.isin(pd.date_range(start='2025-07-17 00:00:00', end='2025-07-17 21:00:00', freq='h'))]

print(df_spot[df_spot['Price'].isna()])
print(f"Nombre de valeurs NaN dans df_spot après remplacement : {df_spot['Price'].isna().sum()}")

Empty DataFrame
Columns: [Price]
Index: []
Nombre de valeurs NaN dans df_spot après remplacement : 0


In [44]:
# Sauvegarder le DataFrame df_spot dans un fichier CSV
df_spot.to_csv(os.path.join(output, "prix_spot.csv"), index=True, sep=";")